In [1]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)


In [2]:
!pip install huggingface datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.8 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
      Successfully uninstalled fsspec-2024.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.1+cu121 req

In [3]:
!pip install sentence_transformers

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [4]:
import pandas as pd
from datasets import load_dataset
import numpy as np
from sentence_transformers import SentenceTransformer
import torch
import scipy.spatial

dataset = load_dataset("traversaal-ai-hackathon/hotel_datasets")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/5997 [00:00<?, ? examples/s]

In [5]:
df=pd.DataFrame(dataset['train'])
df.head()

,hotel_name,hotel_description,review_title,review_text,rate,tripdate,hotel_url,hotel_image,price_range,rating_value,review_count,street_address,locality,country
0,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,"An exceptional boutique hotel, great value for...",None,NaN,February 2020,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye
1,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,You can’t get better than this.,None,NaN,March 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye
2,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,Exceeds all expectations,None,NaN,March 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye
3,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,"Great Location, Fantastic Accommodations",None,NaN,August 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye
4,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,Perfection. It is all in the details.,None,NaN,June 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye


In [6]:
df.rate.value_counts()

,count
rate,
5.0,3435
4.0,622
1.0,308
3.0,288
2.0,214


###check how many null 'hotel_description' each hotel has, show me the 100 entry


In [7]:

df.groupby('hotel_name')['rate'].apply(lambda x: x.isnull().sum()).sort_values(ascending=False)[1:40]


,rate
hotel_name,
"Crowne Plaza Istanbul - Florya, an IHG hotel",40
Mula Hotel,30
Ramada by Wyndham Istanbul Old City,30
Hotel Emblem San Francisco,30
Hotel Yasmak Sultan,30
Hyatt Place San Francisco Downtown,30
Elite World Grand İstanbul Basın Ekspres,30
InterContinental Istanbul,30
LUMA Hotel San Francisco,30


###check number of null hotel descriptions

In [8]:

df['hotel_description'].isnull().sum()


800

###how many reviews each hotel has, we can infer that each hotel has been represented equally

In [9]:
df['hotel_name'].value_counts()

,count
hotel_name,
Romance Istanbul Hotel,40
Motto by Hilton New York City Chelsea,40
The Bryant Park Hotel,40
Pod Times Square,40
Tempo by Hilton New York Times Square,40
...,...
San Francisco Marriott Fisherman's Wharf,40
Hilton San Francisco Financial District,40
The Donatello,40


###now we want to see the hotel rate distribution

In [10]:
hotel_rates = df.groupby('hotel_name')['rate'].first().to_dict()
len(hotel_rates)

150

In [11]:
# prompt: check unique values of hotel_rates and how many times each value repeated

unique_rates, rate_counts = np.unique(list(hotel_rates.values()), return_counts=True)

for rate, count in zip(unique_rates, rate_counts):
  print(f"Rate: {rate}, Count: {count}")


Rate: 1.0, Count: 14
Rate: 2.0, Count: 4
Rate: 3.0, Count: 4
Rate: 4.0, Count: 15
Rate: 5.0, Count: 111
Rate: nan, Count: 2


###fill the missing rate values, we can take it from other reviews since each hotel has the same review

In [12]:
# Define the function to fill missing rates
def fill_rate(row):
    if pd.isna(row['rate']):
        return hotel_rates.get(row['hotel_name'], row['rate'])  # Return the matched rate or leave it as NaN if no match
    else:
        return row['rate']

# Apply the function to each row in the DataFrame
df['rate'] = df.apply(fill_rate, axis=1)

In [13]:
df['rate'].isnull().sum()

80

### city distribution

In [14]:
df['locality'].value_counts()

,count
locality,
Istanbul,1200
San Francisco,1200
London,1200
Paris,1200
New York City,1197


##numerical to text representaion

In [15]:

# Assuming df is your DataFrame
# Create a mapping for rating_value
rating_value_map = {
    5.0: 'Very Satisfied Customer',
    4.5: 'Satisfied Customer',
    4.0: 'Moderately Satisfied Customer',
    3.5: 'Neutral Customer',
    3.0: 'Dissatisfied Customer'
}

# Create a mapping for price_range
price_range_map = {
    '$ (Based on Average Nightly Rates for a Standard Room from our Partners)': 'Economical',
    '$$ (Based on Average Nightly Rates for a Standard Room from our Partners)': 'Moderate',
    '$$$ (Based on Average Nightly Rates for a Standard Room from our Partners)': 'Expensive'
}

# Create a mapping for rate
rate_map = {
    5.0: '5 Stars',
    4.0: '4 Stars',
    3.0: '3 Stars',
    2.0: '2 Stars',
    1.0: '1 Star',
    float('nan'): 'not known how many stars'
}

# Apply the mappings to the DataFrame
df['rating_value'] = df['rating_value'].map(rating_value_map)
df['price_range'] = df['price_range'].map(price_range_map)
df['rate'] = df['rate'].map(rate_map)

###we see how the data looks like now

In [16]:
df.head()

,hotel_name,hotel_description,review_title,review_text,rate,tripdate,hotel_url,hotel_image,price_range,rating_value,review_count,street_address,locality,country
0,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,"An exceptional boutique hotel, great value for...",None,5 Stars,February 2020,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,Economical,Very Satisfied Customer,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye
1,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,You can’t get better than this.,None,5 Stars,March 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,Economical,Very Satisfied Customer,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye
2,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,Exceeds all expectations,None,5 Stars,March 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,Economical,Very Satisfied Customer,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye
3,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,"Great Location, Fantastic Accommodations",None,5 Stars,August 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,Economical,Very Satisfied Customer,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye
4,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,Perfection. It is all in the details.,None,5 Stars,June 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,Economical,Very Satisfied Customer,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye


###download the model

In [19]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("nomic-ai/nomic-embed-text-v1", trust_remote_code=True)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/128 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/70.9k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.03k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

##embedding the hotel data

In [20]:
def create_combined_embedding(row):
    # Use empty strings for None values
    description = (row['hotel_description'] or "").strip()
    rate = (row['rate'] or "").strip()
    price_range = (row['price_range'] or "").strip()

    combined_text = f"hotel_description: {description}; hotel star rate: {rate}; price range: {price_range}"
    embedding = model.encode(combined_text)
    return embedding.tolist()  # Convert the embedding to a list

# Assuming df is your DataFrame
df["hotel_combined_embedding"] = df.apply(create_combined_embedding, axis=1)

In [21]:
df.head()

,hotel_name,hotel_description,review_title,review_text,rate,tripdate,hotel_url,hotel_image,price_range,rating_value,review_count,street_address,locality,country,hotel_combined_embedding
0,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,"An exceptional boutique hotel, great value for...",None,5 Stars,February 2020,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,Economical,Very Satisfied Customer,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye,"[-0.03911564126610756, 0.051101792603731155, -..."
1,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,You can’t get better than this.,None,5 Stars,March 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,Economical,Very Satisfied Customer,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye,"[-0.03911564126610756, 0.051101792603731155, -..."
2,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,Exceeds all expectations,None,5 Stars,March 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,Economical,Very Satisfied Customer,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye,"[-0.03911564126610756, 0.051101792603731155, -..."
3,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,"Great Location, Fantastic Accommodations",None,5 Stars,August 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,Economical,Very Satisfied Customer,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye,"[-0.03911564126610756, 0.051101792603731155, -..."
4,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,Perfection. It is all in the details.,None,5 Stars,June 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,Economical,Very Satisfied Customer,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye,"[-0.03911564126610756, 0.051101792603731155, -..."


In [22]:
df['rating_value']

,rating_value
0,Very Satisfied Customer
1,Very Satisfied Customer
2,Very Satisfied Customer
3,Very Satisfied Customer
4,Very Satisfied Customer
...,...
5992,Moderately Satisfied Customer
5993,Moderately Satisfied Customer
5994,Moderately Satisfied Customer
5995,Moderately Satisfied Customer


### embedding customer rating

In [23]:
customer_rating = df['rating_value'].tolist()
customer_rating_embeddings = model.encode(customer_rating, show_progress_bar=True)#A
print(f"customer_rating_embeddings shape: {customer_rating_embeddings.shape}")

Batches:   0%|          | 0/188 [00:00<?, ?it/s]

customer_rating_embeddings shape: (5997, 768)


In [24]:
len(customer_rating_embeddings)

5997

###storing embedding in dataframe

In [25]:
# Convert embeddings to a list of lists (each embedding is a list)
embedding_list = [embedding.tolist() for embedding in customer_rating_embeddings]

# Add the embeddings as a new column to the original DataFrame
df['rating_value_embedding'] = embedding_list

In [26]:
df.head()

,hotel_name,hotel_description,review_title,review_text,rate,tripdate,hotel_url,hotel_image,price_range,rating_value,review_count,street_address,locality,country,hotel_combined_embedding,rating_value_embedding
0,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,"An exceptional boutique hotel, great value for...",None,5 Stars,February 2020,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,Economical,Very Satisfied Customer,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye,"[-0.03911564126610756, 0.051101792603731155, -...","[-0.027432873845100403, 0.09858879446983337, -..."
1,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,You can’t get better than this.,None,5 Stars,March 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,Economical,Very Satisfied Customer,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye,"[-0.03911564126610756, 0.051101792603731155, -...","[-0.027432873845100403, 0.09858879446983337, -..."
2,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,Exceeds all expectations,None,5 Stars,March 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,Economical,Very Satisfied Customer,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye,"[-0.03911564126610756, 0.051101792603731155, -...","[-0.027432873845100403, 0.09858879446983337, -..."
3,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,"Great Location, Fantastic Accommodations",None,5 Stars,August 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,Economical,Very Satisfied Customer,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye,"[-0.03911564126610756, 0.051101792603731155, -...","[-0.027432873845100403, 0.09858879446983337, -..."
4,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,Perfection. It is all in the details.,None,5 Stars,June 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,Economical,Very Satisfied Customer,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye,"[-0.03911564126610756, 0.051101792603731155, -...","[-0.027432873845100403, 0.09858879446983337, -..."


###saving df

In [27]:
df.to_csv('df.csv', index=True)

##loading the dataframe with review embeddings

In [32]:
df_new = pd.read_csv('last_df.csv')

In [33]:
df_new.head()

,Unnamed: 0,hotel_name,hotel_description,review_title,review_text,rate,tripdate,hotel_url,hotel_image,price_range,rating_value,review_count,street_address,locality,country,hotel_combined_embedding,rating_value_embedding,review_embedding
0,0,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,"An exceptional boutique hotel, great value for...",NaN,5 Stars,February 2020,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,Economical,Very Satisfied Customer,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye,"[-0.03911564126610756, 0.051101792603731155, -...","[-0.027432873845100403, 0.09858879446983337, -...","[-1.1348376274108887, 1.802743673324585, -5.20..."
1,1,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,You can’t get better than this.,NaN,5 Stars,March 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,Economical,Very Satisfied Customer,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye,"[-0.03911564126610756, 0.051101792603731155, -...","[-0.027432873845100403, 0.09858879446983337, -...","[2.011662721633911, -0.022828077897429466, -4...."
2,2,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,Exceeds all expectations,NaN,5 Stars,March 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,Economical,Very Satisfied Customer,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye,"[-0.03911564126610756, 0.051101792603731155, -...","[-0.027432873845100403, 0.09858879446983337, -...","[0.13323014974594116, 1.3691492080688477, -5.7..."
3,3,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,"Great Location, Fantastic Accommodations",NaN,5 Stars,August 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,Economical,Very Satisfied Customer,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye,"[-0.03911564126610756, 0.051101792603731155, -...","[-0.027432873845100403, 0.09858879446983337, -...","[-1.0223147869110107, 1.7390284538269043, -5.4..."
4,4,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,Perfection. It is all in the details.,NaN,5 Stars,June 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,Economical,Very Satisfied Customer,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye,"[-0.03911564126610756, 0.051101792603731155, -...","[-0.027432873845100403, 0.09858879446983337, -...","[1.8795020580291748, 0.4809856116771698, -4.66..."


###fix double naming for turkiye

In [34]:
df_new['country'].unique()

array(['Turkiye', 'Türkiye', 'United States', 'United Kingdom', 'France'],
      dtype=object)

In [35]:
df_new['country'] = df_new['country'].replace('Türkiye', 'Turkey')

In [36]:
df_new['country'].unique()

array(['Turkiye', 'Turkey', 'United States', 'United Kingdom', 'France'],
      dtype=object)

In [37]:
df_new.head()

,Unnamed: 0,hotel_name,hotel_description,review_title,review_text,rate,tripdate,hotel_url,hotel_image,price_range,rating_value,review_count,street_address,locality,country,hotel_combined_embedding,rating_value_embedding,review_embedding
0,0,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,"An exceptional boutique hotel, great value for...",NaN,5 Stars,February 2020,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,Economical,Very Satisfied Customer,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye,"[-0.03911564126610756, 0.051101792603731155, -...","[-0.027432873845100403, 0.09858879446983337, -...","[-1.1348376274108887, 1.802743673324585, -5.20..."
1,1,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,You can’t get better than this.,NaN,5 Stars,March 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,Economical,Very Satisfied Customer,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye,"[-0.03911564126610756, 0.051101792603731155, -...","[-0.027432873845100403, 0.09858879446983337, -...","[2.011662721633911, -0.022828077897429466, -4...."
2,2,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,Exceeds all expectations,NaN,5 Stars,March 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,Economical,Very Satisfied Customer,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye,"[-0.03911564126610756, 0.051101792603731155, -...","[-0.027432873845100403, 0.09858879446983337, -...","[0.13323014974594116, 1.3691492080688477, -5.7..."
3,3,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,"Great Location, Fantastic Accommodations",NaN,5 Stars,August 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,Economical,Very Satisfied Customer,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye,"[-0.03911564126610756, 0.051101792603731155, -...","[-0.027432873845100403, 0.09858879446983337, -...","[-1.0223147869110107, 1.7390284538269043, -5.4..."
4,4,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,Perfection. It is all in the details.,NaN,5 Stars,June 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,Economical,Very Satisfied Customer,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye,"[-0.03911564126610756, 0.051101792603731155, -...","[-0.027432873845100403, 0.09858879446983337, -...","[1.8795020580291748, 0.4809856116771698, -4.66..."


In [38]:
!python -m spacy download en_core_web_trf


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.4/457.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.3/236.3 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.6/731.6 kB 50.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


###functions for implementing NER

In [39]:
import spacy
import pandas as pd

nlp = spacy.load("en_core_web_trf")

# Function to extract city name from the query
def get_city_name(query):
    text_query = nlp(query)
    for city in text_query.ents:
        if city.label_ == "GPE":
            return city.text.lower()
    return None

# Function to filter DataFrame by location
def filter_by_loc(query):
    city_name = get_city_name(query)
    if city_name in df_new['locality'].str.lower().unique():
        filtered_df = df_new[df_new['locality'].str.lower() == city_name.lower()]
        return filtered_df
    else:
        return df_new

In [40]:

query = "cheap hotel in Istanbul"

query_embedding = model.encode(query)
query_embedding.shape

(768,)

In [41]:
import torch.nn as nn
import torch
import ast



# Function to calculate similarity score
def get_similarity_score(row, query_embedding):
    similarity = nn.CosineSimilarity(dim=0)  # Use dim=0 for 1D tensors

    # Safely evaluate string representations of lists
    rating_value_embedding = torch.tensor(ast.literal_eval(row['rating_value_embedding']))
    hotel_combined_embedding = torch.tensor(ast.literal_eval(row['hotel_combined_embedding']))
    review_embedding = torch.tensor(ast.literal_eval(row['review_embedding']))

    sim1 = similarity(rating_value_embedding, query_embedding)
    sim2 = similarity(hotel_combined_embedding, query_embedding)
    sim3 = similarity(review_embedding, query_embedding)

    return sim1.item() + sim2.item() + sim3.item()

# Main function to process the query and return results
def process_query(query):

    query_embedding = model.encode(query)

    # Filter DataFrame by location
    filtered_data = filter_by_loc(query)

    # Convert query_embedding to a tensor if it is not already
    query_embedding_tensor = torch.tensor(query_embedding)

    # Apply the similarity function to the filtered DataFrame
    filtered_data['similarity_score'] = filtered_data.apply(lambda row: get_similarity_score(row, query_embedding_tensor), axis=1)



    top_similar = filtered_data.sort_values('similarity_score', ascending=False).head(1)


    hotel_name = top_similar['hotel_name'].values[0]
    hotel_description = top_similar['hotel_description'].values[0]
    hotel_rate = top_similar['rate'].values[0]
    hotel_price_range = top_similar['price_range'].values[0]
    hotel_review = top_similar['review_title'].values[0]
    hotel_city = top_similar['locality'].values[0]
    hotel_country = top_similar['country'].values[0]

    # Format the output

    result = "Here's the most similar hotel we found:\n"
    result += "-" * 30 + "\n"
    result += f"Hotel Name: {hotel_name}\n"
    result += f"City: {hotel_city}\n"
    result += f"Country: {hotel_country}\n"
    result += f"Star Rating: {hotel_rate}\n"
    result += f"Price Range: {hotel_price_range}\n"

    return result


# here is the returned df

result_df = process_query(query)

result_df


<ipython-input-41-3dc3a4376de0>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['similarity_score'] = filtered_data.apply(lambda row: get_similarity_score(row, query_embedding_tensor), axis=1)


"Here's the most similar hotel we found:\n------------------------------\nHotel Name: YOTEL Istanbul Airport Hotel (Landside)\nCity: Istanbul\nCountry: Turkey\nStar Rating: 4 Stars\nPrice Range: Moderate\n"

In [ ]:

# Extract the relevant information from the top similar hotel
hotel_name = top_similar['hotel_name'].values[0]
hotel_description = top_similar['hotel_description'].values[0]
hotel_rate = top_similar['rate'].values[0]
hotel_price_range = top_similar['price_range'].values[0]
hotel_review = top_similar['review_title'].values[0]
hotel_city = top_similar['locality'].values[0]
hotel_country = top_similar['country'].values[0]

# Print the information in an ordered fashion
print("query: ",query)
print("-" * 30)
print("Here's the most similar hotel we found:")
print("-" * 30)
print(f"Hotel Name: {hotel_name}")
print("City:", hotel_city)
print("Country:", hotel_country)
# print(f"Description: {hotel_description}")
print(f"Star Rating: {hotel_rate}")
print(f"Price Range: {hotel_price_range}")




In [43]:
!pip install gradio


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 10.6 MB/s eta 0:00:00
  Attempting uninstall: tomlkit
    Found existing 

#gradio implementation

In [44]:
import gradio as gr

ui = gr.Interface(
    fn=process_query,
    inputs=gr.Textbox(label="Query", placeholder="Enter your query"),
    outputs="text",
    title="Hotel Similarity Finder",
    description="Enter a query to find similar hotels."
)

ui.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://997391ed04183baca0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


# end of notebook

---

